In [11]:
%pip install httpx duckduckgo-search numexpr --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
import re
import traceback
from duckduckgo_search import DDGS
import httpx
import numexpr as ne
from src.simpleChatBot import SimpleChatBot as ChatBot
from prompts.prompt_react_agent import prompt


In [13]:
# This code is adapted from Simon Willison https://til.simonwillison.net/llms/python-react-pattern

ACTION_RE=re.compile(r"^Action:\s*([\w]+)\s*:\s*(.+)$", re.DOTALL)


def query(question, max_turns=5):
    i=0
    bot=ChatBot(prompt)
    next_prompt=question
    label="BOT RESPONSE:"
    while i<max_turns:
        i+=1
        try:
            result=bot(next_prompt)
            print(label, result)

            # Extract actions from result
            actions=[ACTION_RE.match(action.strip()) for action in result.split("\n") if
                     ACTION_RE.match(action.strip())]

            if actions:
                action, action_input=actions[0].groups()
                action_input=action_input.strip()

                if action not in known_actions:
                    raise ValueError(f"Unknown action: {action}: {action_input}")

                print(f" -- Running {action} {action_input}")
                observation=known_actions[action](action_input)
                print("Observation:", observation)

                next_prompt=f"Observation: {observation}"
            else:
                return  # No action found, exit loop

        except Exception as e:
            print(f"Error: {e}")
            traceback.print_exc()
            return


def websearch(q, max_results=1):
    try:
        results=DDGS().text(q, max_results=max_results)
        # results[0]["body"] = "I don't know the answer, I am sorry."
        print("WEB SEARCH RESULTS>>>", results)
        if results:
            return results[0]["body"]
        else:
            return "No relevant information found."
    except Exception as e:
        print(f"Search error: {e}")
        return "Error: Could not retrieve search results."


def wikisearch(q):
    try:
        response=httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "prop": "extracts",
            "exintro": True,
            "titles": q,
            "format": "json"
        })
        if response.status_code==200:
            pages=response.json().get("query", {}).get("pages", {})
            if pages:
                return list(pages.values())[0].get("extract", "No information found.")
            else:
                return "Error: No search results found."
        else:
            return f"Error: Failed to fetch data. Status code: {response.status_code}"
    except httpx.RequestError as e:
        return f"Error: Request failed. {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"


def calculate(expression):
    try:
        result=ne.evaluate(expression)
        print("CALCULATED>>>", result)
        return result
    except Exception as e:  #Catches the broader set of Numexpr errors
        print(f"Calculation error: {e}")
        return "Error: Invalid calculation expression."


def ask_user(q):
    return input(f"Clarification needed, INPUT>>>: {q}")


known_actions={
    "search": websearch,
    "wikisearch": wikisearch,
    "calculate": calculate,
}


In [14]:
query("What is the capital of england?")

BOT RESPONSE: Thought: I need to find the capital city of England.  A quick internet search should suffice.
Action: search: capital of england
PAUSE

 -- Running search capital of england
WEB SEARCH RESULTS>>> [{'title': 'London - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/London', 'body': 'London [c] is the capital and largest city [d] of both England and the United Kingdom, with a population of 8,866,180 in 2022. [2] Its wider metropolitan area is the largest in Western Europe, with a population of 14.9 million. [7] London stands on the River Thames in southeast England, at the head of a 50-mile (80 km) tidal estuary down to the North Sea, and has been a major settlement for ...'}]
Observation: London [c] is the capital and largest city [d] of both England and the United Kingdom, with a population of 8,866,180 in 2022. [2] Its wider metropolitan area is the largest in Western Europe, with a population of 14.9 million. [7] London stands on the River Thames in southeast England

In [16]:
query("What is (2 + 2)*10?")

BOT RESPONSE: Thought: This is a simple arithmetic problem. I can use the `calculate` action to solve it.
Action: calculate: (2 + 2) * 10
PAUSE

 -- Running calculate (2 + 2) * 10
CALCULATED>>> 40
Observation: 40
BOT RESPONSE: Answer: 40



In [17]:
query("What is the population of Paris times 2 plus the population of London")

BOT RESPONSE: Thought: I need to find the populations of Paris and London, then perform the calculation.  I'll use a search engine to find the populations.

Action: search: population of Paris, population of London

PAUSE

 -- Running search population of Paris, population of London
WEB SEARCH RESULTS>>> [{'title': 'London vs. Paris - Comparison of sizes - Size Explorer', 'href': 'https://www.size-explorer.com/en/compare/cities/london/paris/', 'body': "Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third-most populous of any city in Europe and accounts for 13.4% of the UK population. London's urban area is the third most populous in Europe, after Moscow and Paris, with 9,787,426 inhabitants at the 2011 census."}]
Observation: Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third-most populous of any city in Europe and accounts for 13.4% of the UK population. London's urban area is the thi